# Kundenabwanderung

Kundenabwanderung ist ein branchenübergreifendes Problem.
Häufig wird sehr viel Energie in die Gewinnung von Kunden gesteckt, aber das Halten der Kunden wird vernachlässigt.
Mit einer Prognose zur Kundenabwanderung können diejenigen Kunden identifiziert werden, welche gegebenenfalls abwandern würden.
Diese kann das Unternehmen dann mit besonderen Angeboten oder anderweitigen Aufmerksamkeiten vielleicht zurückgewinnen.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

Nun laden wir die Datei `CustomerChurn.xlsx` herunter.
Der Code dafür ist bereits vorgegeben.
IBM stellt den Datensatz
[in einem Blogbeitrag](https://community.ibm.com/community/user/businessanalytics/blogs/steven-macko/2019/07/11/telco-customer-churn-1113)
vor.

In [ ]:
import os
import requests
import shutil

file_name = "CustomerChurn.xlsx"
if file_name not in os.listdir("."):
    print(f"Lade Datei '{file_name}' herunter")
    r = requests.get(
        "https://public.dhe.ibm.com/software/data/sw-library/cognos/mobile/C11/data/CustomerChurn.xlsx",
        stream=True
    )
    if r.status_code == 200:
        with open(file_name, 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)
else:
    print(f"Datei '{file_name}' ist bereits heruntergeladen")

## Einlesen der Daten

Das Modul `pandas` liefert die Möglichkeit, Excel-Dateien mit der Funktion `read_excel()` einzulesen.

In [ ]:
df = pd.read_excel("CustomerChurn.xlsx")
df

Es wurden verschiedene Skalen für Attribute vorgestellt.
Die Methode `df.info()` zeigt an, welcher Typ innerhalb von pandas ermittelt worden ist.
Eine kurze technische Erklärung der Typen gibt es
[in der Dokumentation von pandas](https://pandas.pydata.org/docs/user_guide/basics.html#dtypes).

<span style="color:blue">(a) 
    Führen Sie die nachfolgende Anweisung aus und interpretieren Sie das Ergebnis.
    Welche Spalten werden als nominalskaliert erkannt (`object`), sollten aber eine andere Skala haben?
</span>

In [ ]:
df.info()

## Umwandlung in korrekte Typen

Nun möchten wir aus dem Text in der Spalte `Total Charges` Zahlenwerte gewinnen, die wir weiter verwenden können.

In [ ]:
df["Total Charges"] = pd.to_numeric(df['Total Charges'], errors='coerce')
df

<span style="color:blue">(b)
    Führen Sie die nachfolgende Anweisung aus und interpretieren Sie das Ergebnis.
    Was hat sich verändert?
</span>

In [ ]:
df.info()

Spoiler Alert: Später im Jupyter Notebook möchten wir noch Maschinelles Lernen verwenden.
Deswegen wollen wir schon mal alle "Yes"-Einträge in Einsen und alle "No"-Einträge in Nullen umwandeln.

In [ ]:
for attr in ["Phone Service", "Online Security", "Senior Citizen", "Partner", "Dependents",
             "Device Protection", "Paperless Billing", "Streaming Movies", "Streaming TV",
             "Tech Support", "Churn", "Multiple Lines", "Online Backup"]:
    df[attr] = df[attr].map({'Yes': 1, 'No': 0, 1: 1, 0: 0})

df

Das Umwandeln der Vertragsarten und der Bezahlmethode sind komplexer.
Deswegen überspringen wir das für diese Übung und löschen die Werte einfach.

In [ ]:
df.drop(columns=["Payment Method", "Contract", "Internet Service"], inplace=True)
df

## Wahrscheinlichkeit, dass ein Kunde abspringt

Das Modul `pandas` stellt eine Reihe von deskriptiven Analysewerkzeugen zur Verfügung, bspw. die Berechnung von Mittelwerten, Varianzen und Standardabweichungen einzelner Spalten.
Weiterführende Informationen dazu sind in der
[Dokumentation von pandas zu deskriptiven Statistiken](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html#computations-descriptive-stats)
zu finden.

<span style="color:blue">(c)
    Führen Sie die nachfolgende Anweisung aus und interpretieren Sie das Ergebnis.
    Was hat es mit der empirischen Wahrscheinlichkeit zu tun, dass ein Kunde abspringt?
</span>

In [ ]:
df['Churn'].mean()

## Eine ML-basierte Prognose

Nun soll ein Algorithmus entwickelt werden, der vorhersagt, ob ein bestimmter Kunde abwandern wird.
Im nächsten Schritt teilen wir die Daten auf:
`X` enthält die Eingabewerte, die uns etwas über das Ergebnis verraten sollen,
und `y` enthält das Ziel, was wir erlernen wollen.
Die Spalte `y` wollen wir in Zukunft mit ML prognostizieren können.

In [ ]:
X = df.drop(
    ['Customer ID', 'LoyaltyID', 'Churn'],  # Nenne alle Spalten, die nicht zum Lernen verwendet werden dürfen/sollen
    axis=1
).values
y = df['Churn'].values  # Das zu erlernene Ziel

Nun erstellen wir einen Entscheidungsbaum.
Weitere Infos hier finden Sie
[in der Dokumentation von scikit-learn](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html).

In [ ]:
from sklearn import tree
clf_dt = tree.DecisionTreeClassifier(max_depth=3)

In einem nächsten Schritt wird der Entscheidungsbaum anhand der Trainingsdaten trainiert.
Dabei erlernt er den (bzw. einen) Zusammenhang zwischen `X` und `y`.

In [ ]:
clf_dt.fit(X, y)

Gerade eben kam es vermutlich zu einer Fehlermeldung (genauer gesagt, einem `ValueError`).
Der Entscheidungsbaum von scikit-learn braucht nicht nur überall Zahlenwerte, sondern es dürfen auch keine Daten fehlen.
Deswegen verwerfen wir kurzerhand alle Zeilen, in denen wir fehlende Werte haben.

In [ ]:
df.dropna(axis=1, how='any', inplace=True)
df

Nun rufen wir das Lernverfahren erneut auf:

In [ ]:
X = df.drop(['Customer ID', 'LoyaltyID', 'Churn'], axis=1).values
y = df['Churn'].values

clf_dt.fit(X, y)

Diesmal sollte es geklappt haben.
Falls nicht, haben Sie vermutlich bei Aufgabe (d) etwas übersehen.

Jetzt, da der Entscheidungsbaum fertig ist, sollten wir überprüfen, in wie viel Prozet der Fälle der Entscheidungsbaum richtig liegt.

In [ ]:
clf_dt.score(X, y)

In [ ]:
print("Wahrscheinlichkeit, dass der Kunde abwandert", df['Churn'].mean() * 100, "%")
print("Wahrscheinlichkeit, dass der Kunde nicht abwandert", (1 - df['Churn'].mean()) * 100, "%")

<span style="color:blue">(d)
    Setzen Sie das Ergebnis in den Kontext, was wir bereits allein aufgrund des arithmetischen Mittels wissen.
    Würden Sie dieses Modell in der Praxis einsetzen?
</span>

Der folgende Code visualisierung den Entscheidungsbaum:

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
tree.plot_tree(
    clf_dt,
    ax=ax,
    feature_names=df.drop(['Customer ID', 'LoyaltyID', 'Churn'], axis=1).columns
)
plt.show()

Die Boxen in der untersten Zeile stellen die Blätter da, die anderen Boxen sind die Knoten des Baumes.
In diesen Boxen ist die jeweils oberste Zeile die Bedingung, die überprüft wird.
Ist die Bedingung wahr, so wird der linke Pfad gewählt und ist die Bedigung falsch, so wird der rechte Pfad gewählt.

- Die Zeile mit dem `gini`-Wert gibt die Verteilung von abgewanderten zu beim Kunden verbliebenen Kunden beim jeweiligen Knoten bzw. Blatt an.
- Die Zeile mit dem `sample`-Wert gibt an, wie viele Zeilen der Daten (ohne Testdaten) den jeweiligen Knoten bzw. das jeweilige Blatt erreicht haben.
- Die Zeile mit dem `value`-Wert gibt an, wie viele Passagiere (ohne Testdaten) des jeweiligen Knotens bzw. Blattes als Kunden geblieben sind (1. Wert) bzw. abgewandert sind (2. Wert).
- Bei den Blättern gibt der größere `value`-Wert an, welche Prognose der Algorithmus für neue Daten trifft.

<span style="color:blue">(e)
    Ist diese Art von Entscheidungsbaum zu erwarten gewesen?
    Gibt es Unterscheidungen und Prognosen, die überraschend sind? Woran liegt dies?
</span>